In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlencode
import concurrent.futures

API_KEY = 'e082cc9aab123618a435d30d8c353b03'

In [12]:
%%time

SUCHWORT = "Fussballhandschuhe"

def extract_one_review(url, userlist, reviewlist, starlist, datelist):
    params = {'api_key': API_KEY, 'url': url}
    
    for _ in range(NUM_RETRIES):
        try:
            response = requests.get('http://api.scraperapi.com/', params=urlencode(params))
            if response.status_code in [200, 404]:
                ## escape for loop if the API returns a successful response
                break
        except requests.exceptions.ConnectionError:
            response = ''
    
    if response.status_code == 200:
    
        soup = BeautifulSoup(response.text, 'html.parser')
        soupreviews = soup.find("div", class_="reviews-content")
    
        if soupreviews != None:
            reviewblock = soupreviews.find_all("div", {"data-hook": "review"})
            for item in reviewblock:
                date = item.find("span", {"data-hook": "review-date"})
                if item != None and date.get_text().startswith('Rezension aus Deutschland'):
                    user = item.find("span", class_="a-profile-name")
                    if user != None:
                        userlist.append(user.get_text())
                    else:
                        userlist.append(' ')
                    body = item.find("span", {"data-hook": "review-body"})
                    if body != None:
                        reviewlist.append(body.get_text())
                    else:
                        reviewlist.append( '')
                    star = item.find("span", class_="a-icon-alt")
                    if star != None:
                        starlist.append(star.get_text().split('von')[0][:-3])
                    else:
                        starlist.append(' ')
                    date = item.find("span", {"data-hook": "review-date"})
                    if date != None:
                        datelist.append(date.get_text().split('vom')[1][1:])
                    else:
                        datelist.append(' ')

        weiter = soup.find("li", class_="a-last")
        if weiter != None:
            links = weiter.find("a")
        else:
            links = None
        if links is not None:
            link_url = 'https://www.amazon.de' + links["href"]
        else:
            link_url = ''
        return {'user': userlist, 'reviews': reviewlist, 'bewertungen': starlist, 'reviewdates': datelist, 'nexturl': link_url}
    
def extract_reviews(url):
    userlist = []
    reviewlist = []
    starlist = []
    datelist = []
    
    response = extract_one_review(url, userlist, reviewlist, starlist, datelist)
    link_url = response["nexturl"]
    
    while link_url != '':
        response = extract_one_review(link_url, list(response["user"]), list(response["reviews"]), list(response["bewertungen"]), 
                                      list(response["reviewdates"]))
        link_url = response["nexturl"]
    
    return {'user': response['user'], 'reviews': response['reviews'], 'bewertungen': response['bewertungen'], 
            'reviewdates': response['reviewdates']}

def extract_details(url):
    params = {'api_key': API_KEY, 'url': url}
    
    for _ in range(NUM_RETRIES):
        try:
            response = requests.get('http://api.scraperapi.com/', params=urlencode(params))
            if response.status_code in [200, 404]:
                ## escape for loop if the API returns a successful response
                break
        except requests.exceptions.ConnectionError:
            response = ' '
            
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        titeltext = soup.find("span", id="productTitle")
        if titeltext != None:
            titeltext = titeltext.get_text()
        else:
            titeltext = ' '
    
        features = soup.find("div", id="feature-bullets")
        if features != None:
            features  = features.get_text()
        else:
            features = ' '
      
        reviewurl = soup.find("a", {"data-hook": "see-all-reviews-link-foot"})
    
        if reviewurl != None:
            reviewurl = 'https://www.amazon.de' + reviewurl["href"]
    
        response = extract_reviews(reviewurl)
    
        erglist.append({'titel': titeltext, 'features': features, 'user': response['user'], 
            'reviews': response['reviews'], 'bewertungen': response['bewertungen'], 'reviewdates': response['reviewdates']})
        


NUM_RETRIES = 3
NUM_THREADS = 50

url_s = 'https://www.amazon.de/s?k=' + SUCHWORT


params = {'api_key': API_KEY, 'url': url_s}


for _ in range(NUM_RETRIES):
        try:
            response = requests.get('http://api.scraperapi.com/', params=urlencode(params))
            if response.status_code in [200, 404]:
                ## escape for loop if the API returns a successful response
                break
        except requests.exceptions.ConnectionError:
            response = ''

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    suchergebnisse = soup.find_all("div", {"data-component-type": "s-search-result"})

    produkt_urls = []
    for i in range(len(suchergebnisse)):
        produktitel = suchergebnisse[i].find("a")
        if produktitel != None:
            produkt_urls.append('https://www.amazon.de' + produktitel["href"])

erglist = []

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
    executor.map(extract_details, produkt_urls)

suchdf = pd.DataFrame()
for i in range(len(erglist)):
    temp = pd.DataFrame.from_dict(erglist[i])
    suchdf = pd.concat([suchdf, temp], axis=0, ignore_index=True)

suchdf.to_csv(SUCHWORT + '.csv', index=False, sep=";")

CPU times: total: 1min 36s
Wall time: 10min 39s
CPU times: total: 27.4 s
Wall time: 54.9 s
CPU times: total: 35.2 s
Wall time: 58.6 s
CPU times: total: 35 s
Wall time: 1min 29s
